In [ ]:
!pip install transformers[sentencepiece]
!pip install accelerate


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 83.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 118.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 8.7 MB/s eta 0:00:00


## Fine tune a model on the training data


In [ ]:
import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

# Same as before
checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'classifier.weight', 'pre_classifier.

In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.0 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

dataset = load_dataset("imdb")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

### As in the tutorial we use the Dataset.map function to keep the flexibility of this structure

In [ ]:
def tokenize_function(data):
    return tokenizer(data['text'], padding=True, truncation=True)

tokenized_data = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
# Used for auto padding inside batches

In [ ]:
from transformers import TrainingArguments
# As we train for only one epoch we use a eval strategy and save strategy to step
# We could have used better batch size for both training an validation for the training to be faster
training_args = TrainingArguments("test-trainer", num_train_epochs=1, evaluation_strategy='steps',save_strategy='steps',)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
500,0.402700,0.283655
1000,0.330900,0.346634
1500,0.309600,0.258714
2000,0.282600,0.265815
2500,0.255100,0.253278
3000,0.268200,0.218988


TrainOutput(global_step=3125, training_loss=0.30718263916015626, metrics={'train_runtime': 3817.4333, 'train_samples_per_second': 6.549, 'train_steps_per_second': 0.819, 'total_flos': 3311684966400000.0, 'train_loss': 0.30718263916015626, 'epoch': 1.0})

 ## Below is the model fine tuned on accuracy

In [ ]:
from sklearn.metrics import accuracy_score
def compute_accuracy(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = accuracy_score(labels, preds)
    return {"accuracy": accuracy}


In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # Directory where checkpoints and logs will be saved
    num_train_epochs=1,              # Number of training epochs
    per_device_train_batch_size=16,  # Batch size for training
    per_device_eval_batch_size=128,   # Batch size for evaluation/validation
    save_strategy='steps',           # Save checkpoints after each steps
    evaluation_strategy='steps',     # Evaluate and save checkpoints after each steps
)

trainer = Trainer(
    model=model,                     # Your model
    args=training_args,              # Training arguments
    train_dataset=tokenized_data["train"],     # Your training dataset
    eval_dataset=tokenized_data['test'],       # Your evaluation/validation dataset
    compute_metrics=compute_accuracy  # Custom evaluation metric function
)

# Train the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
500,0.171700,0.287950,0.914800
1000,0.163800,0.302487,0.909920
1500,0.187900,0.210215,0.927600


TrainOutput(global_step=1563, training_loss=0.17787070344521003, metrics={'train_runtime': 2588.6098, 'train_samples_per_second': 9.658, 'train_steps_per_second': 0.604, 'total_flos': 3311684966400000.0, 'train_loss': 0.17787070344521003, 'epoch': 1.0})

## Evaluate the model in term of accuracy on the test data.

In [ ]:
!pip install evaluate
import numpy as np
from evaluate import evaluator
import evaluate


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 7.3 MB/s eta 0:00:00


In [ ]:
predictions = trainer.predict(tokenized_data["test"])
print(predictions.predictions.shape, predictions.label_ids.shape)

(25000, 2) (25000,)


In [ ]:
task_evaluator = evaluator("text-classification")


eval_results = task_evaluator.compute(
    model_or_pipeline=model,
    data=tokenized_data["test"],
    label_mapping={"NEGATIVE": 0, "POSITIVE": 1},
    metric=evaluate.combine(["accuracy", "recall", "precision", "f1"]),
    tokenizer=tokenizer)

print(eval_results)

##  For at least 2 samples which have been wrongly classified in the test set, try explaining why the model could have been wrong.

## What are the advantages and inconvenient of using this model in production compared to the naive Bayes we implemented in the first part of the course? And compared to a recurrent model like an RNN or an LSTM?

### This king of model can achieve several task in a more precise way than the others we have seen before, but is longer to compute, espcecially compared to naive bayes

### In another hand, RNN and LSTM can reach high accuracy score, but take a really long time to train, were much of the calculation can be parallelized in a transformer.

### Furthermore being able to fine tune already trained model save us huge amount of time (and data to find) at training.